# Export Predictions to CSV (for R)

In [19]:
import os
import matplotlib.pyplot as plt
import yaml
#from ct_classifier/ct_classifier.train import create_dataloader, load_model       # NOTE: since we're using these functions across files, it could make sense to put them in e.g. a "util.py" script.
import torch
import pandas as pd
import numpy as np
from torch import softmax as softmax

In [20]:
#os.getcwd()
os.chdir('/home/magali/ct_classifier/ct_classifier')

In [21]:
from train import load_model
from train import create_dataloader

## Parameters

In [26]:
config = '/home/magali/ct_classifier/configs/exp_resnet18_sex.yaml'
split = 'val'
BEST_MODEL_PATH = '/home/magali/ct_classifier/best_models/sex_basic_run2_60.pt'


## Load Data

In [27]:
# load config
print(f'Using config "{config}"')
cfg = yaml.safe_load(open(config, 'r'))


# setup entities
dl_val = create_dataloader(cfg, split='val')

# load model
#model = torch.load(BEST_MODEL_PATH)
#model = load_model(cfg)[0] #takes first model, refer to bets model here
#model = load_model(cfg)[BEST_MODEL_PATH]
#BEST_MODEL_PATH = '/home/magali/ct_classifier/best_models/sex_basic_run1_epoch54.pt'
model_instance, start_epoch = load_model(cfg, model_path=BEST_MODEL_PATH)



Using config "/home/magali/ct_classifier/configs/exp_resnet18_sex.yaml"
val number of images 1540 labels 1540 images covered 1540
Loading model from /home/magali/ct_classifier/best_models/sex_basic_run2_60.pt


In [ ]:
#model_instance, start_epoch = load_model(cfg, model_path=BEST_MODEL_PATH)

#state = torch.load(BEST_MODEL_PATH, map_location='cpu')
#model_instance.load_state_dict(state)



## Visualize

This is up to you to figure out now. :)

In [31]:
print(type(model_instance))
#print(model)
#print(prediction)

<class 'model.CustomResNet18'>


In [32]:
#device = cfg['device']
#model.to(device)

model_instance.eval()

CustomResNet18(
  (feature_extractor): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affin

In [34]:
all_pred_labels = []
all_labels = []
all_img_path = []

def tuple_to_list(obj):
    if isinstance(obj, tuple):
        return [tuple_to_list(item) for item in obj]
    return obj

with torch.no_grad():               # don't calculate intermediate gradient steps: we don't need them, so this saves memory and is faster
        for idx, (data, labels, img_path) in enumerate(dl_val):

            # put data and labels on device
            #data, labels = data.to(device), labels.to(device)
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


            print(data.shape)
            #print(model.classifier.device)
            print(data.device)
            print(device)
            # forward pass
            prediction = model_instance(data)
            #print(prediction[0])
            #print(img_path[0])

            # If you want to track labels and paths as well, you can continue to do that
            pred_label = prediction
            all_pred_labels = all_pred_labels + pred_label.cpu().tolist()
            print(all_pred_labels)
            all_labels = all_labels + labels.tolist()
            print(all_labels)

            # Convert tuple to numpy array
            #img_path_arr = np.array(img_path)
            #img_path_lst = tuple_to_list(img_path)

            #all_img_path = all_img_path + img_path_arr.tolist()

            # Flatten the img_path tuple
            img_path_lst = tuple_to_list(img_path)
            all_img_path.extend(img_path_lst)

            # Print sizes for debugging
            print(f"Iteration {idx}: all_pred_labels size: {len(all_pred_labels)}, all_labels size: {len(all_labels)}, all_img_path size: {len(all_img_path)}")

            
        # After the loop, print or return the summed predictions
        print(all_pred_labels, all_labels, all_img_path)




torch.Size([128, 3, 224, 224])
cpu
cuda
[[1.7823926210403442, -0.988662600517273], [1.5261512994766235, -0.6987549066543579], [1.044891357421875, -0.7075648307800293], [1.4820212125778198, -0.8055243492126465], [1.4315155744552612, -1.078636884689331], [1.4293227195739746, -0.1621074378490448], [1.5499354600906372, -0.2927209436893463], [1.553908109664917, -0.8665285110473633], [2.127190113067627, -0.6580382585525513], [0.961045503616333, 0.1464129388332367], [-0.07756173610687256, 0.6908117532730103], [0.12327158451080322, 0.6390780210494995], [0.6704825162887573, -0.13740995526313782], [0.6831004619598389, -0.23508688807487488], [0.11060522496700287, 0.22931024432182312], [-0.820579469203949, 1.4830350875854492], [0.9595093131065369, -1.2243742942810059], [1.3975616693496704, -1.1822885274887085], [1.7531118392944336, -0.9869785308837891], [-1.2152477502822876, 2.311615467071533], [0.42616310715675354, 0.036453746259212494], [0.5095597505569458, 0.4482864439487457], [1.95053350925445

In [35]:
print(all_labels)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 

Create dataframes with predictions, labels and image id

In [36]:
label_df= pd.DataFrame(all_labels, columns=['Labels'])
pred_df = pd.DataFrame(all_pred_labels, columns=['pred_female', 'pred_male'])
img_id_df = pd.DataFrame(all_img_path, columns=['Image path'])


In [37]:
print(pred_df)

      pred_female  pred_male
0        1.782393  -0.988663
1        1.526151  -0.698755
2        1.044891  -0.707565
3        1.482021  -0.805524
4        1.431516  -1.078637
...           ...        ...
1535    -0.087545   0.779994
1536     0.744894  -0.660017
1537     2.102342  -1.062880
1538     0.887922   0.230736
1539     0.801753  -0.056981

[1540 rows x 2 columns]


In [38]:
print(img_id_df)

                                             Image path
0     /home/magali/CV4Ecology-summer-school/Prototyp...
1     /home/magali/CV4Ecology-summer-school/Prototyp...
2     /home/magali/CV4Ecology-summer-school/Prototyp...
3     /home/magali/CV4Ecology-summer-school/Prototyp...
4     /home/magali/CV4Ecology-summer-school/Prototyp...
...                                                 ...
1535  /home/magali/CV4Ecology-summer-school/Prototyp...
1536  /home/magali/CV4Ecology-summer-school/Prototyp...
1537  /home/magali/CV4Ecology-summer-school/Prototyp...
1538  /home/magali/CV4Ecology-summer-school/Prototyp...
1539  /home/magali/CV4Ecology-summer-school/Prototyp...

[1540 rows x 1 columns]


In [39]:
result = pd.concat([label_df, pred_df, img_id_df], axis=1, ignore_index=False)
print(result)

      Labels  pred_female  pred_male  \
0          0     1.782393  -0.988663   
1          0     1.526151  -0.698755   
2          0     1.044891  -0.707565   
3          0     1.482021  -0.805524   
4          0     1.431516  -1.078637   
...      ...          ...        ...   
1535       0    -0.087545   0.779994   
1536       0     0.744894  -0.660017   
1537       0     2.102342  -1.062880   
1538       0     0.887922   0.230736   
1539       0     0.801753  -0.056981   

                                             Image path  
0     /home/magali/CV4Ecology-summer-school/Prototyp...  
1     /home/magali/CV4Ecology-summer-school/Prototyp...  
2     /home/magali/CV4Ecology-summer-school/Prototyp...  
3     /home/magali/CV4Ecology-summer-school/Prototyp...  
4     /home/magali/CV4Ecology-summer-school/Prototyp...  
...                                                 ...  
1535  /home/magali/CV4Ecology-summer-school/Prototyp...  
1536  /home/magali/CV4Ecology-summer-school/Prototyp...

convert predictions to with softmax

In [40]:
# Define a function to apply torch.softmax to a list of scores
#def apply_softmax(scores):
#    tensor_scores = torch.tensor(scores)
#    softmax_scores = torch.softmax(tensor_scores, dim=0).tolist()  # Using dim=0 for 1D tensor
#    return softmax_scores

# Apply the function to the 'prediction' column
#result['pred_female_softmax'] = result['pred_female'].apply(torch.softmax)
#result['pred_male_softmax'] = result['pred_male'].apply(apply_softmax)

# Convert DataFrame columns to a tensor
tensor_predictions = torch.tensor(result[['pred_female', 'pred_male']].values)

# Compute softmax along the last dimension (across columns for each row)
softmax_tensor = torch.softmax(tensor_predictions, dim=1)

# Convert the tensor back to a DataFrame
df_softmax = pd.DataFrame(softmax_tensor.numpy(), columns=['pred_female_softmax', 'pred_male_softmax'])

# Concatenate the two DataFrames along axis=1
result = pd.concat([result, df_softmax], axis=1)

print(result)


      Labels  pred_female  pred_male  \
0          0     1.782393  -0.988663   
1          0     1.526151  -0.698755   
2          0     1.044891  -0.707565   
3          0     1.482021  -0.805524   
4          0     1.431516  -1.078637   
...      ...          ...        ...   
1535       0    -0.087545   0.779994   
1536       0     0.744894  -0.660017   
1537       0     2.102342  -1.062880   
1538       0     0.887922   0.230736   
1539       0     0.801753  -0.056981   

                                             Image path  pred_female_softmax  \
0     /home/magali/CV4Ecology-summer-school/Prototyp...             0.941092   
1     /home/magali/CV4Ecology-summer-school/Prototyp...             0.902464   
2     /home/magali/CV4Ecology-summer-school/Prototyp...             0.852262   
3     /home/magali/CV4Ecology-summer-school/Prototyp...             0.907840   
4     /home/magali/CV4Ecology-summer-school/Prototyp...             0.924850   
...                                    

In [41]:
print(result)

      Labels  pred_female  pred_male  \
0          0     1.782393  -0.988663   
1          0     1.526151  -0.698755   
2          0     1.044891  -0.707565   
3          0     1.482021  -0.805524   
4          0     1.431516  -1.078637   
...      ...          ...        ...   
1535       0    -0.087545   0.779994   
1536       0     0.744894  -0.660017   
1537       0     2.102342  -1.062880   
1538       0     0.887922   0.230736   
1539       0     0.801753  -0.056981   

                                             Image path  pred_female_softmax  \
0     /home/magali/CV4Ecology-summer-school/Prototyp...             0.941092   
1     /home/magali/CV4Ecology-summer-school/Prototyp...             0.902464   
2     /home/magali/CV4Ecology-summer-school/Prototyp...             0.852262   
3     /home/magali/CV4Ecology-summer-school/Prototyp...             0.907840   
4     /home/magali/CV4Ecology-summer-school/Prototyp...             0.924850   
...                                    

In [16]:
result.to_csv('../predictions/predictions_sex_basic.csv', index=True)
